In [1]:
import pandas as pd
import numpy as np
import joblib
import math
from keras.models import load_model

import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
# 전처리 끝낸 데이터
df_train = joblib.load('df_train.pkl')
df_test = joblib.load('df_test.pkl')


# RFC

In [3]:
# 이동평균을 만들어서 해보자

In [4]:
df = df_train.copy()

In [5]:
# df_train = df_train.drop(['year', 'month', 'day', 'time', 'minute', 'stn_id','vis1'],axis=1)

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [7]:
# # 결측치와 무한대 값 처리
# df.replace([np.inf, -np.inf], np.nan, inplace=True)
# df.dropna(inplace=True)

In [34]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

# 데이터 불러오기
# df_train = pd.read_csv('path_to_your_dataset.csv') # 필요시 사용

# 특성과 타겟 변수 분리
X = df_train[['year', 'month', 'time','ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'dew_point']]
y = df_train['class']

# 클래스 값을 0, 1, 2, 3으로 변환
y = y - 1

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123,  stratify=y)

# XGBoost 모델 생성
rf_model = RandomForestClassifier(n_estimators=100)
# 모델 학습
rf_model.fit(X_train, y_train)

# 예측
y_pred = rf_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')
print(f'Confusion Matrix:\n{conf_matrix}')

Accuracy: 0.9959131518364416
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.65      0.70       173
           1       0.59      0.33      0.42       171
           2       0.47      0.17      0.25       219
           3       1.00      1.00      1.00     94131

    accuracy                           1.00     94694
   macro avg       0.71      0.54      0.59     94694
weighted avg       0.99      1.00      1.00     94694

Confusion Matrix:
[[  112    22     4    35]
 [   31    56    17    67]
 [    1    11    38   169]
 [    3     6    21 94101]]


In [35]:
# H, F, M 계산
H = np.sum((y_test == y_pred) & (y_test != 3))  # Hits
F = np.sum((y_test != y_pred) & (y_pred != 3))  # False Alarms
M = np.sum((y_test != y_pred) & (y_pred == 3))  # Misses

# CSI 계산
CSI = H / (H + F + M)

# 결과 출력
print(f"Calculated CSI: {CSI}")

Calculated CSI: 0.3473861720067454


# 모델 적용

In [36]:
# df_test = df_test.drop(['year', 'month', 'day', 'time', 'minute', 'stn_id','vis1'],axis=1)

In [37]:
# 특성과 타겟 변수 분리
X_test = df_test[['year', 'month', 'time','ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'dew_point']]
y_test = df_test['class']

# 클래스 값을 0, 1, 2, 3으로 변환
y_test = y_test - 1

# 예측
y_pred = rf_model.predict(X_test)

In [38]:
# 예측된 값을 df_test에 적용
df_test['class'] = y_pred + 1  # 원래 클래스 값으로 복원

In [39]:
df_test.head()

,year,month,day,time,minute,stn_id,ws10_deg,ws10_ms,ta,re,hm,sun10,ts,dew_point,vis1,class
210241,3,1,1,0,0,4,0.741039,0.412844,0.437931,0.0,0.472579,0.0,0.086162,0.409787,NaN,4
210242,3,1,1,0,10,4,0.778272,0.376147,0.434483,0.0,0.487748,0.0,0.080940,0.413802,NaN,4
210243,3,1,1,0,20,4,0.799944,0.376147,0.431034,0.0,0.508751,0.0,0.078329,0.420326,NaN,4
210244,3,1,1,0,30,4,0.763545,0.403670,0.434483,0.0,0.499417,0.0,0.080940,0.418821,NaN,4
210245,3,1,1,0,40,4,0.754932,0.449541,0.434483,0.0,0.507585,0.0,0.080940,0.422334,NaN,4


In [40]:
pred_E = df_test.copy()

In [33]:
pred_E.to_csv('pred_E_time.csv')

# 모델 합치기

In [ ]:
#### 여기까지하고 지희한테 보내기

In [107]:
# fog_test.csv부르기
df_original = pd.read_csv("fog_test.csv", index_col=1)
df_original['fog_test.class'].replace(-99, float('nan'), inplace=True)

In [108]:
# pred_A = pd.read_csv('C:\\Users\\Hong_PC\\Documents\\카카오톡 받은 파일\\240371.csv', index_col=0)
pred_C = pd.read_csv('C:\\Users\\Hong_PC\\Documents\\카카오톡 받은 파일\\240371_2번지역.csv', index_col=0)

In [109]:
# 지역 주의, 예측된 class 값만 df_original에 반영
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'A', 'fog_test.class'] = pred_A['fog_test.class'].values
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'B', 'fog_test.class'] = pred_B['class'].values
df_original.loc[df_original['fog_test.stn_id'].str[0] == 'C', 'fog_test.class'] = pred_C['class'].values
# df_original.loc[df_original['fog_test.stn_id'].str[0] == 'D', 'fog_test.class'] = pred_D['class'].values
df_original.loc[df_original['fog_test.stn_id'].str[0] == 'E', 'fog_test.class'] = pred_E['class'].values


In [110]:
df_original

,Unnamed: 0,fog_test.month,fog_test.day,fog_test.time,fog_test.minute,fog_test.stn_id,fog_test.ws10_deg,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,fog_test.class
fog_test.year,,,,,,,,,,,,,,
L,1,1,1,0,0,AI,329.5,0.6,-2.5,0.0,87.5,0.0,-1.6,NaN
L,2,1,1,0,10,AI,321.8,1.2,-2.5,0.0,88.2,0.0,-1.6,NaN
L,3,1,1,0,20,AI,0.4,0.4,-2.5,0.0,88.6,0.0,-1.7,NaN
L,4,1,1,0,30,AI,323.6,0.7,-2.6,0.0,88.7,0.0,-1.6,NaN
L,5,1,1,0,40,AI,208.4,0.2,-2.6,0.0,89.0,0.0,-1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L,262796,12,31,23,10,ED,338.8,6.1,4.5,0.0,71.3,0.0,1.9,4.0
L,262797,12,31,23,20,ED,335.2,5.8,4.6,0.0,71.4,0.0,1.9,4.0
L,262798,12,31,23,30,ED,337.1,5.3,4.5,0.0,72.1,0.0,1.8,4.0


In [111]:
# 혹시 결측값 있으면 4로 채움 
df_original['fog_test.class'].fillna(4, inplace=True)

In [112]:
df_original['fog_test.class'].value_counts()

4.0    262519
1.0       205
3.0        50
2.0        26
Name: fog_test.class, dtype: int64

In [102]:
df_original.to_csv('240371.csv', index=True)